In [1]:
import os
import pandas as pd

In [2]:
mydir = "/home/barbera/Downloads/runXXX-20210107-SC-Tom/processing/20211104-align-C-region/"
myfiles = [x for x in os.listdir(mydir) if x.endswith("-easy-import.txt")]
myfiles = [mydir + x for x in myfiles]
myfiles

['/home/barbera/Downloads/runXXX-20210107-SC-Tom/processing/20211104-align-C-region/AMC-TC-x001_S0_L001.assembled-nomatch-IGHC_CH12_human-easy-import.txt']

In [3]:
get_barcode = lambda x: x.split("-")[0]
get_contig = lambda x: x.split("_")[2]
get_cname = lambda x: x.split("|")[1].split("*")[0]

In [4]:
def readAlignment(f):
    df = pd.read_csv(f, sep="\t", header=None)
    df.columns = ["acc", "field2", "ref"]
    df["barcode"] = [x for x in map(get_barcode, df["acc"])]
    df["contig"] = [x for x in map(get_contig, df["acc"])]
    df["cregion"] = [x for x in map(get_cname, df["ref"])]
    return(df)

In [5]:
df = readAlignment(myfiles[0])
for myfile in myfiles[1:]:
    df = pd.concat([df, readAlignment(myfile)])
df.tail()

,acc,field2,ref,barcode,contig,cregion
4762,CTTAGGAAGATGCGAC-1_contig_2,0,AJ390237|IGHG3*06|CH1|CH2,CTTAGGAAGATGCGAC,2,IGHG3
4763,TTGCCGTGTCTCTTAT-1_contig_4,0,AJ390238|IGHG3*07|CH1|CH2,TTGCCGTGTCTCTTAT,4,IGHG3
4764,CTTACCGAGCGATGAC-1_contig_2,0,AJ390247|IGHG3*11|CH1|CH2,CTTACCGAGCGATGAC,2,IGHG3
4765,AAAGCAAGTGGACGAT-1_contig_1,0,AJ390254|IGHG3*14|CH1|CH2,AAAGCAAGTGGACGAT,1,IGHG3
4766,TCTGGAAGTGGCAAAC-1_contig_2,0,AJ390254|IGHG3*14|CH1|CH2,TCTGGAAGTGGCAAAC,2,IGHG3


In [6]:
concatNames = lambda x: ",".join(list(set(x)))
df_count_per_barcode = df.groupby("barcode").agg({'contig': "nunique", 'cregion': ["nunique", concatNames]})
df_count_per_barcode = df_count_per_barcode.reset_index()
df_count_per_barcode.head()

barcode  contig cregion         
                    nunique nunique <lambda>
0  AAACCTGAGGCTAGGT       1       1     IGHM
1  AAACCTGAGTGCGATG       1       1     IGHM
2  AAACCTGCAATCTACG       1       1     IGHM
3  AAACCTGGTTCAGTAC       1       1     IGHM
4  AAACCTGGTTCATGGT       1       1     IGHM

In [7]:
df_count_per_barcode.columns = [' '.join(col).strip() for col in df_count_per_barcode.columns.values]
df_count_per_barcode = df_count_per_barcode.rename(columns={'cregion <lambda>': 'cregion name'})
df_count_per_barcode.head()

,barcode,contig nunique,cregion nunique,cregion name
0,AAACCTGAGGCTAGGT,1,1,IGHM
1,AAACCTGAGTGCGATG,1,1,IGHM
2,AAACCTGCAATCTACG,1,1,IGHM
3,AAACCTGGTTCAGTAC,1,1,IGHM
4,AAACCTGGTTCATGGT,1,1,IGHM


In [8]:
df_count_per_barcode = df_count_per_barcode.sort_values(['cregion nunique', 'contig nunique'], ascending=False)
df_count_per_barcode.head()

,barcode,contig nunique,cregion nunique,cregion name
549,AGATTGCAGCTGCAAG,5,3,"IGHM,IGHD,IGHA1"
1763,CGTGTCTTCACAAACC,4,3,"IGHM,IGHD,IGHA1"
4119,TTTACTGTCGATCCCT,3,3,"IGHM,IGHD,IGHA1"
1781,CGTTCTGCACAACGCC,5,2,"IGHM,IGHD"
401,ACGGCCACACTGTCGG,4,2,"IGHM,IGHA2"


In [9]:
# How many cregions were assigned per cell?
df_count_per_barcode.groupby('cregion nunique').agg({'barcode': "nunique"})

,barcode
cregion nunique,
1,3903
2,260
3,3


In [10]:
# Show the entries with 4 assigned c regions
df_count_per_barcode[df_count_per_barcode["cregion nunique"] == 4]

,barcode,contig nunique,cregion nunique,cregion name


In [11]:
# Show the entries with 3 assigned c regions
df_count_per_barcode[df_count_per_barcode["cregion nunique"] == 3]

,barcode,contig nunique,cregion nunique,cregion name
549,AGATTGCAGCTGCAAG,5,3,"IGHM,IGHD,IGHA1"
1763,CGTGTCTTCACAAACC,4,3,"IGHM,IGHD,IGHA1"
4119,TTTACTGTCGATCCCT,3,3,"IGHM,IGHD,IGHA1"


In [12]:
# Show the entries with 2 assigned c regions
df_count_per_barcode[df_count_per_barcode["cregion nunique"] == 2]

,barcode,contig nunique,cregion nunique,cregion name
1781,CGTTCTGCACAACGCC,5,2,"IGHM,IGHD"
401,ACGGCCACACTGTCGG,4,2,"IGHM,IGHA2"
593,AGCGTCGAGCAACGGT,4,2,"IGHM,IGHA1"
2735,GGATGTTCACATTAGC,4,2,"IGHD,IGHG2"
3846,TGGCGCAAGTAGCCGA,4,2,"IGHM,IGHD"
43,AAAGTAGCATCCTAGA,3,2,"IGHM,IGHD"
356,ACGAGGAGTGCAGACA,3,2,"IGHM,IGHD"
536,AGAGTGGGTACAAGTA,3,2,"IGHM,IGHD"
679,AGGTCCGCAGCATACT,3,2,"IGHM,IGHD"
854,ATGTGTGCATGGGAAC,3,2,"IGHM,IGHD"


In [13]:
df_count_per_barcode.to_excel("count-c-region.xlsx")
print("Wrote count-c-region.xlsx to disk")

Wrote count-c-region.xlsx to disk
